# **Data Pre-Processing (User and EDA)**

# Importing Libraries

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip install gender-guesser

In [ ]:
import pandas as pd
import spacy
import scipy
import numpy as np
import sys
sys.path.append('/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages')
import gender_guesser.detector as gender
from unidecode import unidecode

# Importing Functions

In [ ]:
from data_cleaning_functions.clean_text_eda import clean_text_eda
from saving_loading_functions.saving_file import saving_file
from saving_loading_functions.saving_file_json import saving_file_json
from saving_loading_functions.loading_file import loading_file
from saving_loading_functions.loading_file_json import loading_file_json

# Declaring Filepaths

In [ ]:
raw_filepath = 'data/raw/'
uuid_filepath = 'data/processed/uuid_dataframes/'
tokenised_filepath = 'data/processed/tokenised_sentences/'
user_filepath = 'data/processed/user_data/'
adhoc_filepath = 'data/processed/adhoc_fixes/'

# Loading Datasets from S3 Bucket

In [ ]:
# loading files from bucket
df = loading_file(uuid_filepath, 'all_bank_scraped_data_filled_dates_uuid.csv', 1) # review dataset
user_df_dig = loading_file(raw_filepath, 'users_data_digital_bank_10082022.csv', 0) # user dataset for digital banks
user_df_conven_n26 = loading_file_json(raw_filepath, 'user_data_conventional_and_n26_18082022.json', 0) # user dataset for conventional and n26 banks
cc_df = loading_file(raw_filepath, 'country_code.csv', 0) # country codes

# reformatting long abbreviated countries
cc_df.loc[cc_df.loc[:, "Name"] == "United Arab Emirates", "Name"] = "UAE"
cc_df.loc[cc_df.loc[:, "Name"] == "United Kingdom", "Name"] = "UK"
cc_df.loc[cc_df.loc[:, "Name"] == "United States", "Name"] = "USA"

cc_df = cc_df[["Code", "Name"]] # swapping order of country code and name

# 1) Review Dataset - Removing Stopwords, Punctuation and Formatting Date

In [ ]:
def text_clean(df):
  """
    Takes a dataset of reviews, cleans it, and saves the resulting file
  """
    
    df["cleaned_text"] = ""
    df["yy_mm"] = ""

    # replacing empty texts with titles (i.e. use title instead if no text)
    for i in range(len(df)):
        if pd.isnull(df.loc[i, "text"]):
            df.loc[i, "text"] = df.loc[i, "title"]

        # cleaning text
        df.loc[i, "cleaned_text"] = clean_text_eda(df.loc[i, "text"])

        # formatting date onto new column
        split_date = df.loc[i, "date"].split("-")
        df.loc[i, "yy_mm"] = split_date[0] + "-" + split_date[1]
    
    saving_file(df, uuid_filepath, 'all_bank_scraped_data_filled_dates_uuid_eda.csv')

    return df

df = text_clean(df)

# 2) User Dataset - Merging User Data from Digital Bank and Conventional & N26 Bank

In [ ]:
user_df = user_df_dig.copy()

# concat to correctly and incorrectly formatted users onto single dataframe
user_df = pd.concat([user_df, user_df_conven_n26], ignore_index=True)
user_df.reset_index(drop=True, inplace=True)

# removing duplicated entries based on reviews assigned
print("Number of users: {0}".format(len(user_df)))

user_df["review_details_string"] = user_df.loc[:, "review_details"].astype(str) # converting dict to string
user_df.drop_duplicates(subset=["review_details_string"]) # dropping duplicates based on dictionary of reviews
user_df.reset_index(drop=True, inplace=True)
user_df.drop(['review_details_string'], axis=1, inplace=True) # dropping the created column

print("Number of users after removing duplicates: {0}".format(len(user_df)))

# saving file 
saving_file_json(user_df, user_filepath, 'user_data_all_18082022.json')

user_df

Number of users: 306369
Number of users after removing duplicates: 306369


,user_url,user_name,user_img,review_details
0,/users/5c74a795d33974504c9dcd41,Vilma Aparecida Wilcox,NaN,{'1': {'company_reviewed': ['Wise (formerly Tr...
1,/users/5c2d0b401e51fb7ea5bcdca6,Niklas,NaN,{'1': {'company_reviewed': ['Scribbr - Lektora...
2,/users/61274f64e3101f0012057e67,Ahmed Eshwihdi,NaN,"{'1': {'company_reviewed': ['eToro'], 'user_ra..."
3,/users/6011cfd7cd5140001c4d0a98,Clarke Obama,NaN,"{'1': {'company_reviewed': ['VulkanBet'], 'use..."
4,/users/5dbc18c0cd6c35eb5da3a7cc,Lučan Anamaria,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...","{'1': {'company_reviewed': ['Revolut'], 'user_..."
...,...,...,...,...
306364,/users/5ab18f324de5666d34a4c5c9,Simon,None,"{'1': {'company_reviewed': ['Vax UK'], 'user_r..."
306365,/users/5e643a51dffcf7c90b86ad71,Jennie,None,"{'1': {'company_reviewed': ['FarmaSave'], 'use..."
306366,/users/58c7b3d70000ff000a8154c2,stephen tavener,None,"{'1': {'company_reviewed': ['Saga plc'], 'user..."
306367,/users/5b40ce3c4de5666d3454efd4,customer,None,"{'1': {'company_reviewed': ['Barclays'], 'user..."


# 3) User Dataset - Getting Wrongly Formatted Users

In [ ]:
remaining_users_unformatted = list(set([user_df.loc[i, "user_url"] for i in range(len(user_df)) 
                                        if len(user_df.loc[i, "review_details"]) > 32000]))
rem_users_unformatted_df = pd.DataFrame(data={'remaining_users': remaining_users_unformatted})

# saving file onto bucket
saving_file(rem_users_unformatted_df, adhoc_filepath, 'user_remaining_unformatted.csv')

print("Number of unformatted users: {}".format(len(rem_users_unformatted_df)))

Number of unformatted users: 31


Found 31 users whereby the reviews left by the user was incorrectly scraped. This is due to the user leaving lengthy reviews which exceeded the 32,767 character limit, when storing the dictionary of user reviews, for csv files in a cell. Instead, the reviews left by the user was re-scraped and saved a json file.

In [ ]:
# loading file from bucket
rem_users_unformatted_data_df = loading_file_json(adhoc_filepath, 'users_remaining_unformatted_digital_bank_data.json', 0)
rem_users_unformatted_data_df

,user_url,user_name,user_img,review_details
0,/users/5ddecb4b741d5f95fe2f5aba,bee,None,"{'1': {'company_reviewed': ['Surfshark'], 'use..."
1,/users/5efdfa11f0155f17476797f6,Gaywa46,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...","{'1': {'company_reviewed': ['Trustpilot'], 'us..."
2,/users/5a4219550000ff000af636d7,Faryad,None,"{'1': {'company_reviewed': ['Bitdefender'], 'u..."
3,/users/588f92050000ff000a71fd62,Ross,None,"{'1': {'company_reviewed': ['Ryanair'], 'user_..."
4,/users/586cedfd0000ff000a686625,Mrs Russell,None,"{'1': {'company_reviewed': ['Policy Expert'], ..."
...,...,...,...,...
63,/users/590231e50000ff000a91e83c,Grant Turner,None,"{'1': {'company_reviewed': ['Barclays'], 'user..."
64,/users/4a7021080000640001015538,DL,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...","{'1': {'company_reviewed': ['hungryhouse'], 'u..."
65,/users/56ec19300000ff000a0fdaed,Alex Hedger,None,"{'1': {'company_reviewed': ['KALMARs'], 'user_..."
66,/users/590231e50000ff000a91e83c,Grant Turner,None,"{'1': {'company_reviewed': ['Direct Unlocks'],..."


Rescraped user data of the 31 users, resulting in 68 additional entries (some users with more than 20 reviews)

In [ ]:
user_df_formatted = user_df.copy()

# getting correctly formatted users
for user in remaining_users_unformatted:
    user_df_formatted = user_df_formatted.loc[user_df_formatted.loc[:, "user_url"] != user, :]
user_df_formatted.reset_index(drop=True, inplace=True)

# concat to correctly and incorrectly formatted users onto single dataframe
user_df_formatted = pd.concat([user_df_formatted, rem_users_unformatted_data_df], ignore_index=True)
user_df_formatted.reset_index(drop=True, inplace=True)

# removing rows with missing user_urls as a result of the incorrect formatting
user_df_formatted.dropna(subset=["user_url"], inplace=True)
user_df_formatted.reset_index(drop=True, inplace=True)

# converting to json file and saving file
saving_file_json(user_df_formatted, user_filepath, 'user_data_unformatted_18082022.json')

user_df_formatted

,user_url,user_name,user_img,review_details
0,/users/5c74a795d33974504c9dcd41,Vilma Aparecida Wilcox,NaN,{'1': {'company_reviewed': ['Wise (formerly Tr...
1,/users/5c2d0b401e51fb7ea5bcdca6,Niklas,NaN,{'1': {'company_reviewed': ['Scribbr - Lektora...
2,/users/61274f64e3101f0012057e67,Ahmed Eshwihdi,NaN,"{'1': {'company_reviewed': ['eToro'], 'user_ra..."
3,/users/6011cfd7cd5140001c4d0a98,Clarke Obama,NaN,"{'1': {'company_reviewed': ['VulkanBet'], 'use..."
4,/users/5dbc18c0cd6c35eb5da3a7cc,Lučan Anamaria,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...","{'1': {'company_reviewed': ['Revolut'], 'user_..."
...,...,...,...,...
306323,/users/590231e50000ff000a91e83c,Grant Turner,None,"{'1': {'company_reviewed': ['Barclays'], 'user..."
306324,/users/4a7021080000640001015538,DL,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...","{'1': {'company_reviewed': ['hungryhouse'], 'u..."
306325,/users/56ec19300000ff000a0fdaed,Alex Hedger,None,"{'1': {'company_reviewed': ['KALMARs'], 'user_..."
306326,/users/590231e50000ff000a91e83c,Grant Turner,None,"{'1': {'company_reviewed': ['Direct Unlocks'],..."


Lost 41 (306369 - 306328) rows (not necessarily 41 users) due to reviews being deleted, users deleting their profiles, and/or rows without user_urls as a result of the incorrect formatting

# 4) User Dataset - Compressing Multiple Rows into Single Entry

In [ ]:
user_urls = list(set([url for url in user_df_formatted.loc[:, "user_url"]])) # getting list of unique urls
# user_urls = ['/users/5ddecb4b741d5f95fe2f5aba']
user_names = []
user_imgs = []
review_details = []

# iterating over each unique url
for url in user_urls:
    user_df = user_df_formatted.loc[user_df_formatted.loc[:, "user_url"] == url, :]
    user_dict_1 = ast.literal_eval(str(user_df.iloc[0, -1]))
    
    # storing user_names and user_imgs
    user_names.append(user_df.iloc[0, 1])
    user_imgs.append(user_df.iloc[0, 2])
    
    # reformatting dictionary
    num_pages = len(user_df)
    key_value = 20
    
    if num_pages > 1:
        for i in range(1, num_pages):
            user_next_dict = ast.literal_eval(str(user_df.iloc[i, -1]))
            for j in range(len(user_next_dict.keys())):
                key_value += 1
                user_next_dict[str(key_value)] = user_next_dict.pop(str(j + 1))
            user_dict_1.update(user_next_dict)
    
    # storing dictionary
    review_details.append(user_dict_1)

user_df = pd.DataFrame(data={'user_url': user_urls, 'user_name': user_names, 
                                'user_img': user_imgs, 'review_details': review_details})

# saving file onto bucket
saving_file_json(user_df, user_filepath, 'user_data_formatted_18082022.json')

user_df

,user_url,user_name,user_img,review_details
0,/users/5d35e3daadcf327b959ae832,Corbu Razvan,NaN,"{'1': {'company_reviewed': ['Revolut'], 'user_..."
1,/users/58e53e470000ff000a8a06b1,Wendy Sheriff,NaN,{'1': {'company_reviewed': ['Signomatic.co.uk'...
2,/users/627262a0a38b410012052b9f,Vladimir,NaN,"{'1': {'company_reviewed': ['Goldman Sachs'], ..."
3,/users/6226829ecf47c20012f3368a,Elaine Mowatt,NaN,{'1': {'company_reviewed': ['Wise (formerly Tr...
4,/users/5c4498e1703a76866690188e,Pamela Zenklusen,NaN,"{'1': {'company_reviewed': ['Evri'], 'user_rat..."
...,...,...,...,...
295769,/users/5797bfeb0000ff000a353400,john walder,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",{'1': {'company_reviewed': ['Hatton Garden Met...
295770,/users/5b508fc44de5666d342cbad2,Gerald Rodriguez,NaN,{'1': {'company_reviewed': ['Wise (formerly Tr...
295771,/users/60b7cb4a5b2dac0019a5e278,Christopher Newbold,NaN,"{'1': {'company_reviewed': ['Starling Bank'], ..."
295772,/users/549ab97400006400019fe98f,Gemma,NaN,"{'1': {'company_reviewed': ['LifeSearch'], 'us..."


# 5) Review Dataset - Removing reviews not found in user dataset

In [ ]:
# dropping reviews that are not found in the user dataset
# from review eda dataset
df = loading_file(uuid_filepath, 'all_bank_scraped_data_filled_dates_uuid_eda.csv', 1)
df = df[df['user_url'].isin(user_df.user_url.values)]
df.reset_index(inplace=True, drop=True)
saving_file(df, uuid_filepath, 'all_bank_scraped_data_filled_dates_uuid_eda_final_cleaned.csv')

# from review non-eda dataset
df_2 = loading_file(uuid_filepath, 'all_bank_scraped_data_filled_dates_uuid.csv', 1)
df_2 = df_2[df_2['user_url'].isin(user_df.user_url.values)]
df_2.reset_index(inplace=True, drop=True)
saving_file(df_2, uuid_filepath, 'all_bank_scraped_data_filled_dates_uuid_final_cleaned.csv')

# 6) Review Dataset - Adding YYMMDD Date Format

In [ ]:
df_2["yy_mm_dd"] = df_2["date"].apply(lambda x: x.split("-")[0] + "-" + x.split("-")[1])
saving_file(df_2, uuid_filepath, 'all_bank_scraped_data_final_cleaned_yymmdd.csv')

# 7) User Dataset - Approximating Distribution of Pictures

In [ ]:
user_df = loading_file_json(user_filepath, 'user_data_formatted_18082022.json', 0)

user_df["user_img"] = user_df["user_img"].astype(str)
user_df["img_flag"] = user_df["user_img"].apply(lambda x: 0 if x == "None" else 1)

user_df.head(5)

,user_url,user_name,user_img,review_details,img_flag
0,/users/5d35e3daadcf327b959ae832,Corbu Razvan,None,"{'1': {'company_reviewed': ['Revolut'], 'user_...",0
1,/users/58e53e470000ff000a8a06b1,Wendy Sheriff,None,{'1': {'company_reviewed': ['Signomatic.co.uk'...,0
2,/users/627262a0a38b410012052b9f,Vladimir,None,"{'1': {'company_reviewed': ['Goldman Sachs'], ...",0
3,/users/6226829ecf47c20012f3368a,Elaine Mowatt,None,{'1': {'company_reviewed': ['Wise (formerly Tr...,0
4,/users/5c4498e1703a76866690188e,Pamela Zenklusen,None,"{'1': {'company_reviewed': ['Evri'], 'user_rat...",0


# 8) User Dataset - Approximating Distribution of Gender

In [ ]:
# formatting username to account for accented letters
user_df["user_name"] = user_df["user_name"].astype(str)
user_name_unaccented = [str(unidecode(u'{}'.format(user_df.loc[i, "user_name"])).capitalize()) for i in range(len(user_df))]
user_df["user_name_unaccented"] = user_name_unaccented

# using gender_guesser
user_df["gender_guesser"] = ""
d = gender.Detector()

user_df["gender_guesser"] = user_df["user_name_unaccented"].apply(lambda x: d.get_gender(u"{}".format(x.split(" ")[0])))

saving_file_json(user_df, user_filepath, 'user_data_formatted_gender_picture.json')
    
user_df.head(5)

,user_url,user_name,user_img,review_details,img_flag,user_name_unaccented,gender_guesser
0,/users/5d35e3daadcf327b959ae832,Corbu Razvan,None,"{'1': {'company_reviewed': ['Revolut'], 'user_...",0,Corbu razvan,unknown
1,/users/58e53e470000ff000a8a06b1,Wendy Sheriff,None,{'1': {'company_reviewed': ['Signomatic.co.uk'...,0,Wendy sheriff,female
2,/users/627262a0a38b410012052b9f,Vladimir,None,"{'1': {'company_reviewed': ['Goldman Sachs'], ...",0,Vladimir,male
3,/users/6226829ecf47c20012f3368a,Elaine Mowatt,None,{'1': {'company_reviewed': ['Wise (formerly Tr...,0,Elaine mowatt,female
4,/users/5c4498e1703a76866690188e,Pamela Zenklusen,None,"{'1': {'company_reviewed': ['Evri'], 'user_rat...",0,Pamela zenklusen,female
